# JSONPath vs JMESPath

In [1]:
%pip install beautifulsoup4 lxml jsonpath-ng jmespath jsonpointer

In [ ]:
import json
from bs4 import BeautifulSoup
from lxml import html
from jsonpath_ng import jsonpath, parse
import jmespath
import jsonpointer
from data import json_data, html_doc # data.py


## 1. Dados de Exemplo

In [ ]:

from data import json_data, html_doc # data.py

## 2. Demonstração

### 2.1 CSS Path (BeautifulSoup)

In [5]:
soup = BeautifulSoup(html_doc, 'html.parser')

def run_css_selector(selector: str):
    """
    Executes a CSS selector using BeautifulSoup and prints the results.
    """
    print(f"CSS Selector: '{selector}'")
    elements = soup.select(selector)
    if elements:
        for i, el in enumerate(elements):
            print(f"  Elemento {i+1}: <{el.name}> - Texto: '{el.get_text(strip=True)}'")
    else:
        print("  Nenhum elemento encontrado.")
    print("-" * 30)

run_css_selector('h1')
run_css_selector('#homeLink')
run_css_selector('.menu-item')
run_css_selector('p.texto-destaque')
run_css_selector('div.card span') # Span dentro de div com classe 'card'
run_css_selector('section#conteudo-principal > p') # Pargrafo filho direto de section#conteudo-principal
run_css_selector('input[type="text"]')
run_css_selector('span[data-info="id-1"]')
run_css_selector('div.card:nth-of-type(2) h3') # Título do segundo card

CSS Selector: 'h1'
  Elemento 1: <h1> - Texto: 'Bem-vindo!'
------------------------------
CSS Selector: '#homeLink'
  Elemento 1: <a> - Texto: 'Home'
------------------------------
CSS Selector: '.menu-item'
  Elemento 1: <a> - Texto: 'Home'
  Elemento 2: <a> - Texto: 'Sobre'
------------------------------
CSS Selector: 'p.texto-destaque'
  Elemento 1: <p> - Texto: 'Este é o primeiro parágrafo.'
------------------------------
CSS Selector: 'div.card span'
  Elemento 1: <span> - Texto: 'Detalhe do Item 1'
  Elemento 2: <span> - Texto: 'Detalhe do Item 2'
------------------------------
CSS Selector: 'section#conteudo-principal > p'
  Elemento 1: <p> - Texto: 'Este é o primeiro parágrafo.'
  Elemento 2: <p> - Texto: 'Este é o segundo parágrafo.'
------------------------------
CSS Selector: 'input[type="text"]'
  Elemento 1: <input> - Texto: ''
------------------------------
CSS Selector: 'span[data-info="id-1"]'
  Elemento 1: <span> - Texto: 'Detalhe do Item 1'
--------------------------

### 2.2 XPath (lxml)

In [6]:
tree = html.fromstring(html_doc)

def run_xpath_selector(xpath_expr: str):
    """
    Executes an XPath expression using lxml and prints the results.
    """
    print(f"XPath: '{xpath_expr}'")
    elements = tree.xpath(xpath_expr)
    if elements:
        for i, el in enumerate(elements):
            if hasattr(el, 'tag'): # Check if it's an element node
                print(f"  Elemento {i+1}: <{el.tag}> - Texto: '{el.text_content().strip()}'")
            else: # Likely an attribute or text node
                print(f"  Resultado {i+1}: '{el}'")
    else:
        print("  Nenhum resultado encontrado.")
    print("-" * 30)

run_xpath_selector('//h1')
run_xpath_selector('//a[@id="homeLink"]')
run_xpath_selector('//*[@class="menu-item"]')
run_xpath_selector('//p[@class="texto-destaque"]')
run_xpath_selector('//div[@class="card"]/span')
run_xpath_selector('//section[@id="conteudo-principal"]/p')
run_xpath_selector('//input[@type="text"]')
run_xpath_selector('//span[@data-info="id-1"]')
run_xpath_selector('(//div[@class="card"])[2]/h3') # Segundo card

XPath: '//h1'
  Elemento 1: <h1> - Texto: 'Bem-vindo!'
------------------------------
XPath: '//a[@id="homeLink"]'
  Elemento 1: <a> - Texto: 'Home'
------------------------------
XPath: '//*[@class="menu-item"]'
  Elemento 1: <a> - Texto: 'Home'
  Elemento 2: <a> - Texto: 'Sobre'
------------------------------
XPath: '//p[@class="texto-destaque"]'
  Elemento 1: <p> - Texto: 'Este é o primeiro parágrafo.'
------------------------------
XPath: '//div[@class="card"]/span'
  Elemento 1: <span> - Texto: 'Detalhe do Item 1'
  Elemento 2: <span> - Texto: 'Detalhe do Item 2'
------------------------------
XPath: '//section[@id="conteudo-principal"]/p'
  Elemento 1: <p> - Texto: 'Este é o primeiro parágrafo.'
  Elemento 2: <p> - Texto: 'Este é o segundo parágrafo.'
------------------------------
XPath: '//input[@type="text"]'
  Elemento 1: <input> - Texto: ''
------------------------------
XPath: '//span[@data-info="id-1"]'
  Elemento 1: <span> - Texto: 'Detalhe do Item 1'
--------------------

### 2.3 JSONPath (jsonpath-ng)

In [7]:
def run_jsonpath_query(query: str):
    """
    Executes a JSONPath query and prints the results.
    """
    print(f"JSONPath: '{query}'")
    jsonpath_expr = parse(query)
    matches = jsonpath_expr.find(json_data)
    if matches:
        print(f"  Resultados ({len(matches)}):")
        for i, match in enumerate(matches):
            print(f"    Match {i+1}: {match.value}")
    else:
        print("  Nenhum resultado encontrado.")
    print("-" * 30)

run_jsonpath_query('$.livraria.titulo')
run_jsonpath_query('$.livraria.livros[*].titulo')
run_jsonpath_query('$.livraria.livros[1].autores[0]')


JSONPath: '$.livraria.titulo'
  Resultados (1):
    Match 1: Catálogo de Livros Técnicos
------------------------------
JSONPath: '$.livraria.livros[*].titulo'
  Resultados (4):
    Match 1: Aprendendo JSONPath
    Match 2: Dominando JMESPath
    Match 3: Introdução ao Web Scraping
    Match 4: Python para Iniciantes
------------------------------
JSONPath: '$.livraria.livros[1].autores[0]'
  Resultados (1):
    Match 1: Charlie
------------------------------


#### JSONPath - Limitação para Remodelação/Funções ---

JSONPath (jsonpath-ng) não suporta remodelagem de saída ou funções complexas como 'contains' nativamente na expressão.

Para 'remodelar' o output ou usar 'contains', você precisaria de pós-processamento em Python.

- Ex: Para '{titulo: titulo, primeiroAutor: autores[0]}' você selecionaria os dados e construiria em código.
- Ex: Para 'livros com 'Programação' em tags': Faria o filtro no Python após a seleção inicial.

In [8]:
# run_jsonpath_query('$.livraria.livros[?disponivel=true].titulo') # Filtrando livros disponíveis
# run_jsonpath_query('$.livraria.livros[?(@.preco > 25 && @.editora.nome == "Editora Inovação")].titulo') # Filtro complexo
# run_jsonpath_query('$.usuarios[?isAdmin=true].ultimosAcessos') # Seleciona array de acessos do admin

### 2.4 JMESPath (jmespath)

In [9]:
def run_jmespath_query(query: str):
    """
    Executes a JMESPath query and prints the results.
    """
    print(f"JMESPath: '{query}'")
    result = jmespath.search(query, json_data)
    if result is not None and result != []:
        print(f"  Resultado: {result}")
    else:
        print("  Nenhum resultado encontrado ou resultado vazio.")
    print("-" * 30)

run_jmespath_query('livraria.titulo')
run_jmespath_query('livraria.livros[*].titulo')
run_jmespath_query('livraria.livros[1].autores[0]')
run_jmespath_query('livraria.livros[?disponivel].titulo') # Filtrando livros disponíveis (mais conciso)
run_jmespath_query('livraria.livros[?preco > `25.00` && editora.nome == `Editora Inovação`].titulo') # Filtro complexo
run_jmespath_query('usuarios[?isAdmin][0].ultimosAcessos[-1]') # Último acesso do admin (com índice negativo)


JMESPath: 'livraria.titulo'
  Resultado: Catálogo de Livros Técnicos
------------------------------
JMESPath: 'livraria.livros[*].titulo'
  Resultado: ['Aprendendo JSONPath', 'Dominando JMESPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']
------------------------------
JMESPath: 'livraria.livros[1].autores[0]'
  Resultado: Charlie
------------------------------
JMESPath: 'livraria.livros[?disponivel].titulo'
  Resultado: ['Aprendendo JSONPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']
------------------------------
JMESPath: 'livraria.livros[?preco > `25.00` && editora.nome == `Editora Inovação`].titulo'
  Resultado: ['Aprendendo JSONPath']
------------------------------
JMESPath: 'usuarios[?isAdmin][0].ultimosAcessos[-1]'
  Nenhum resultado encontrado ou resultado vazio.
------------------------------


#### 2.4.1 JMESPath: Vantagem na remodelação (Projections)


In [10]:
run_jmespath_query('livraria.livros[*].{titulo: titulo, primeiroAutor: autores[0]}') # Remodelação!

JMESPath: 'livraria.livros[*].{titulo: titulo, primeiroAutor: autores[0]}'
  Resultado: [{'titulo': 'Aprendendo JSONPath', 'primeiroAutor': 'Alice'}, {'titulo': 'Dominando JMESPath', 'primeiroAutor': 'Charlie'}, {'titulo': 'Introdução ao Web Scraping', 'primeiroAutor': 'David'}, {'titulo': 'Python para Iniciantes', 'primeiroAutor': 'Grace'}]
------------------------------


#### 2.4.2 JMESPath - Vantagem: Remodelação (Projections)

In [11]:
print("\n--- JMESPath - Vantagem: Funções ---")
run_jmespath_query('livraria.livros[?tags && contains(tags, `Programação`)].titulo') # Usando função 'contains' com verificação de existência de 'tags'
run_jmespath_query('livraria.livros[*].autores | [0] | length(@)') # Comprimento do array de autores do primeiro livro
run_jmespath_query('livraria.livros[0].autores | length(@)') # Comprimento do array de autores do primeiro livro
run_jmespath_query('livraria.livros[0].autores | join(`, `, @)') # Juntando autores do primeiro livro


--- JMESPath - Vantagem: Funções ---
JMESPath: 'livraria.livros[?tags && contains(tags, `Programação`)].titulo'
  Resultado: ['Python para Iniciantes']
------------------------------
JMESPath: 'livraria.livros[*].autores | [0] | length(@)'
  Resultado: 2
------------------------------
JMESPath: 'livraria.livros[0].autores | length(@)'
  Resultado: 2
------------------------------
JMESPath: 'livraria.livros[0].autores | join(`, `, @)'
  Resultado: Alice, Bob
------------------------------


### 2.5 JSON Pointer (jsonpointer)

In [12]:
def run_json_pointer(pointer: str):
    """
    Resolves a JSON Pointer and prints the result.
    """
    print(f"JSON Pointer: '{pointer}'")
    try:
        result = jsonpointer.resolve_pointer(json_data, pointer)
        print(f"  Resultado: {result}")
    except jsonpointer.JsonPointerException as e:
        print(f"  Erro: {e}")
    print("-" * 30)

run_json_pointer('') # O documento inteiro
run_json_pointer('/livraria/titulo')
run_json_pointer('/livraria/livros/0') # Primeiro livro
run_json_pointer('/livraria/livros/0/autores/0') # Primeiro autor do primeiro livro
run_json_pointer('/usuarios/1/nome') # Nome do segundo usuário
run_json_pointer('/configuracoes/chave~1com~0slash') # Chave com caracteres especiais (escaped)
run_json_pointer('/caminho/que/nao/existe') # Caminho inexistente

JSON Pointer: ''
  Resultado: {'livraria': {'titulo': 'Catálogo de Livros Técnicos', 'autorPrincipal': 'Dr. Tech', 'livros': [{'titulo': 'Aprendendo JSONPath', 'autores': ['Alice', 'Bob'], 'ano': 2020, 'preco': 25.5, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Dominando JMESPath', 'autores': ['Charlie'], 'ano': 2021, 'preco': 30.0, 'disponivel': False, 'editora': {'nome': 'Editora Global', 'cidade': 'Rio de Janeiro'}}, {'titulo': 'Introdução ao Web Scraping', 'autores': ['David', 'Eve', 'Frank'], 'ano': 2019, 'preco': 20.0, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Python para Iniciantes', 'autores': ['Grace'], 'ano': 2022, 'preco': 35.0, 'disponivel': True, 'tags': ['Programação', 'Iniciante']}]}, 'usuarios': [{'id': 1, 'nome': 'Ana', 'isAdmin': True, 'ultimosAcessos': ['2025-06-19', '2025-06-20']}, {'id': 2, 'nome': 'Bruno', 'isAdmin': False, 'ultimosAcessos': ['2025-06-18']}],